In [1]:
####################################################################################################
########################### 1 - IMPORT DAS BIBLIOTECAS ############################################# 
####################################################################################################

import http.client
import json
import pandas as pd
from datetime import datetime
import numpy as np

from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
####################################################################################################
#################################### 2 - FUNÇÕES ################################################### 
####################################################################################################

def datetime_handler(x):
    if isinstance(x, pd.Timestamp):
        return x.isoformat()
    raise TypeError("Unknown type")
    
#def read_sheets(sheets):
#    result = sheet.values().get(spreadsheetId='',
#                            range=sheets).execute()
#    values_old = result.get('values', [])
#    return len(values_old)


def insert_sheets(df, sheet):
    body = {
    'values': df
    }
    result = service.spreadsheets().values().update(
        spreadsheetId='1P1LbVDyHmyebOUMIa-wyjyJ0fChiHh92p6Hj1FPPItE', range= sheet,
        valueInputOption="USER_ENTERED", body=body).execute()

In [ ]:
####################################################################################################
#################################### 3 - API PANDAPÉ ############################################### 
####################################################################################################

conn = http.client.HTTPSConnection("api.pandape.com.br")
payload = ''
headers = {
  'Authorization': 'Bearer '
}
conn.request("GET", "/v2/requests", payload, headers)
res = conn.getresponse()
data = res.read()

In [4]:
####################################################################################################
############################## 4 - API E SERVIÇOS GOOGLE ########################################### 
####################################################################################################

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [5]:
####################################################################################################
################################## 5 - ETL E INSERT ################################################ 
####################################################################################################

data_dict = json.loads(data)
data_json = json.dumps(data_dict, default = datetime_handler)
read_json = pd.read_json(data_json)
df_request1 = pd.DataFrame(data = read_json)
df_request1 = df_request1.replace(np.nan, '-')



df_request1['creationDate'] = pd.to_datetime(df_request1['creationDate'])

df_request1 = df_request1[df_request1['creationDate'] >= '2022-12-31']

display(df_request1)

list_request1 = df_request1.values.tolist()

for x in range(0, len(list_request1)):
    list_request1[x][4] = str(list_request1[x][4])



insert_sheets(list_request1, 'Request1!A2')

,idRequest,idCompany,requestTitle,vacanciesCount,creationDate,closedDate,sentCandidatesCount,contractedCandidatesCount,idVacancyAssociated,approvedDate
9295,104039,461,MENSAGEIRO,1,2023-01-01 09:09:54.448088100,-,0,0,-,2023-01-02T10:07:15.2288832
9296,104040,461,CONTROLADOR DE ACESSO,1,2023-01-01 09:27:02.487354700,-,0,0,-,2023-01-02T10:07:27.3582212
9297,104041,461,AGENTE DE CONSERVACAO,1,2023-01-01 10:26:29.971000500,-,0,0,-,2023-01-02T10:08:41.6176147
9298,104042,461,CONTROLADOR DE ACESSO,1,2023-01-01 11:45:07.349797500,-,0,0,-,2023-01-02T10:09:21.8540776
9299,104043,461,AGENTE DE CONSERVACAO,1,2023-01-01 11:50:30.762499500,-,0,0,-,2023-01-02T10:08:57.2856523
...,...,...,...,...,...,...,...,...,...,...
20667,417630,461,CONTROLADOR DE ACESSO,1,2025-03-12 07:51:13.951007700,-,0,0,-,-
20668,417637,461,FISCAL DE PATRIMONIO,1,2025-03-12 08:03:49.687664900,-,0,0,-,-
20669,417639,461,FISCAL DE PATRIMONIO,2,2025-03-12 08:09:18.347332800,-,0,0,-,-
20670,417663,461,CONTROLADOR DE ACESSO,1,2025-03-12 08:55:15.322401000,-,0,0,-,-
